# Social Protocol

Protocol想要表征social graph，就需要记录数据及关联关系，以最简单的理解:

UID的所属操作
| UID | Post | Comment  | Mirror |
| ---- | ---- | ---- | ---- |
| u1 | p1,p2 | u2-p1-c1,u2-p1-c2 | u5-p1,... |
| u2 | p1,p2,... | u1-p2-c3,... | |

与UID的交互操作
| UID | Follow | Collect |
| ---- | ---- | ---- |
| u1 | u4,u5 | u4-p3,u5-p1,u2-p3,... |
| u2 | u1 | u7-p1 |

设想在contract中可使用数据结构来记录这个数据关系
```
struct ProfileStruct {
    uint256 pubCount;
    address followNFT;
}

struct PublicationStruct {
    uint256 profileIdPointed;
    uint256 pubIdPointed;
    string contentURI;
    address collectNFT;
}

// profile
mapping(uint256 => address) internal _dispatcherByProfile;  // ProfileId属于who(address)
mapping(uint256 => ProfileStruct) internal _profileById;    // ProfileId对应其属性，pubCount(发帖和评论的index)；followNFT(关注/订阅)
// publish(post&comment) by profile
mapping(uint256 => mapping(uint256 => PublicationStruct)) internal _pubByIdByProfile;  // ProfileId下，pubCount对应的(发帖或评论的详情)

// follow 该如何构造数据结构？
mapping(uint256 => uint256[]) internal _followsByProfile;  // Why not like this?_?
// <== solidity的array不便删除指定位置；而且也不能很好的表征限制订阅的功能

// 常见的设计App中**like**的功能是否需要在合约中实现？
```

## What's the LenHub?

外层`LenHub`主要是规划操作入口，具体的逻辑实现还是通过Library中的合约完成，其中主要包含了两个logic:
- `PublishingLogic.sol`
- `InteractionLogic.sol`

其中`PublishingLogic.sol`中主要实现了`createProfile、createPost、createComment、createMirror`方法；`InteractionLogic.sol`中主要实现了`follow、collect`方法

**从外部看来，Profile将对应一个LenHub的NFT；follow和collect也对应各自的NFT；但是Post、Comment、Mirror是没有形成NFT的**

🤔️ 实现logic进行这样区分的作用是什么？
> 按照LenHub合约中注释的理解，`follow、collect`操作是与profileId进行交互的方法，将其放在一个合约中

🤔️ Follow和Collect使用了NFT，Publishing却不使用NFT？
> 如果说Post、Comment、Mirror是与Profile深度绑定在一起的，没有单独mint NFT的需求
> 
> 那么Follow和Collect使用NFT的好处是可以将其作为融入profile信息圈的一个条件，类似门票的作用，谁有这类NFT就能进入其空间
> 在合约中Follow和Collect的逻辑差异是什么？

## How to defind Storage Struct?

Lens Protocol定义的合约存储变量和结构体

### LenHub Storage Vars

```
// ProfileId => desc
mapping(uint256 => DataTypes.ProfileStruct) internal _profileById;

// PulishingIdx => ProfileId => desc-publication
mapping(uint256 => mapping(uint256 => DataTypes.PublicationStruct)) internal _pubByIdByProfile;

// 递增的profileId
uint256 internal _profileCounter;

// whitelisted
```

### Struct Definded

#### Profile Struct

```
/**
* @notice A struct containing profile data.
*
* @param pubCount The number of publications made to this profile.
* @param followModule The address of the current follow module in use by this profile, can be empty.
* @param followNFT The address of the followNFT associated with this profile, can be empty..
* @param handle The profile's associated handle.
* @param imageURI The URI to be used for the profile's image.
* @param followNFTURI The URI to be used for the follow NFT.
*/
struct ProfileStruct {
    uint256 pubCount;
    address followModule;
    address followNFT;
    string handle;
    string imageURI;
    string followNFTURI;
}
```

📌 表征了账号(profileId)对应的基础属性(handle,imageURI,followNFTURI),publising类型操作的索引(pubCount)
followNFT合约地址是该ProfileId下被follow操作时，给follower mint一个的NFT；followModule合约地址进行限制功能？

#### Publication Struct

```
/**
* @notice A struct containing data associated with each new publication.
*
* @param profileIdPointed The profile token ID this publication points to, for mirrors and comments.
* @param pubIdPointed The publication ID this publication points to, for mirrors and comments.
* @param contentURI The URI associated with this publication.
* @param referenceModule The address of the current reference module in use by this profile, can be empty.
* @param collectModule The address of the collect module associated with this publication, this exists for all publication.
* @param collectNFT The address of the collectNFT associated with this publication, if any.
*/
struct PublicationStruct {
    uint256 profileIdPointed;
    uint256 pubIdPointed;
    string contentURI;
    address referenceModule;
    address collectModule;
    address collectNFT;
}
```

📌 针对pub类型的操作(post,comment,mirror)在这个struct中的配合`profileIdPointed`,`pubIdPointed`和`contentURI`有不同的体现，eg：(0,0,uri) == 发的post; (1,2,uri) == comment; (2,3,'') == mirror

## Profile Owner Functions

> 核心包括`CreateProfile,CreatePost,CreateComment,CreateMirror`操作

#### Create Profile

Function createProfile In `LensHub`

```
function createProfile(DataTypes.CreateProfileData calldata vars)
    external
    override
    whenNotPaused
    returns (uint256)
{
    if (!_profileCreatorWhitelisted[msg.sender]) revert Errors.ProfileCreatorNotWhitelisted();
    unchecked {
        uint256 profileId = ++_profileCounter;
        _mint(vars.to, profileId);
        PublishingLogic.createProfile(
            vars,
            profileId,
            _profileIdByHandleHash,
            _profileById,
            _followModuleWhitelisted
        );
        return profileId;
    }
}
```

📌 给`to`地址mint了一个profile的NFT

createProfile核心逻辑实现在`PublishingLogic`

```
function createProfile(
    DataTypes.CreateProfileData calldata vars,
    uint256 profileId,
    mapping(bytes32 => uint256) storage _profileIdByHandleHash,
    mapping(uint256 => DataTypes.ProfileStruct) storage _profileById,
    mapping(address => bool) storage _followModuleWhitelisted
) external {
    bytes32 handleHash = keccak256(bytes(vars.handle));

    _profileIdByHandleHash[handleHash] = profileId;
    _profileById[profileId].handle = vars.handle;
    _profileById[profileId].imageURI = vars.imageURI;
    _profileById[profileId].followNFTURI = vars.followNFTURI;

    if (vars.followModule != address(0)) {
        _profileById[profileId].followModule = vars.followModule;
    }

    bytes memory followModuleReturnData = _initFollowModule(
        profileId,
        vars.followModule,
        vars.followModuleData,
        _followModuleWhitelisted
    );
}
```

从上面的可以发现，`LensHub`是入口

首先需要判断铸造Profile的地址在白名单中
```
mapping(address => bool) internal _profileCreatorWhitelisted;
```

其中profileId的生成是以合约计数自增(++)的方式，向`to`地址`mint`了指定profileId的NFT

内部再通过其逻辑操作单元`PublishingLogic`中对应地方法进行profile的铸造,同时将具体的信息数据存储到`LensHubStorage`对应的变量中

关于`ProfileStruct`的`pubCount`的表达的是该`profileId`发布操作的计数索引

方法`createPost`的核心logic
```
uint256 pubId = ++_profileById[profileId].pubCount;

_pubByIdByProfile[profileId][pubId].contentURI = contentURI;
```



方法`createComment`的核心logic
```
uint256 pubId = _profileById[vars.profileId].pubCount + 1

uint256 pubCount = _profileById[vars.profileIdPointed].pubCount;
_pubByIdByProfile[vars.profileId][pubId].contentURI = vars.contentURI;
_pubByIdByProfile[vars.profileId][pubId].profileIdPointed = vars.profileIdPointed;
_pubByIdByProfile[vars.profileId][pubId].pubIdPointed = vars.pubIdPointed;

```

方法`createMirror`的核心logic
```
(uint256 rootProfileIdPointed, uint256 rootPubIdPointed, ) = Helpers.getPointedIfMirror(
            vars.profileIdPointed,
            vars.pubIdPointed,
            _pubByIdByProfile
        );

_pubByIdByProfile[vars.profileId][pubId].profileIdPointed = rootProfileIdPointed;
_pubByIdByProfile[vars.profileId][pubId].pubIdPointed = rootPubIdPointed;

```

## Profile Interaction - Follow 

方法`createMirror`的核心logic
```
address followModule = _profileById[profileId].followModule;
address followNFT = _profileById[profileId].followNFT;

if (followNFT == address(0)) {
    followNFT = _deployFollowNFT(profileId);
    _profileById[profileId].followNFT = followNFT;
}

tokenId = IFollowNFT(followNFT).mint(follower);

if (followModule != address(0)) {
    IFollowModule(followModule).processFollow(
        follower,
        profileId,
        followModuleData
    );
}
```

在进行follow操作下[Tx](https://mumbai.polygonscan.com/tx/0x0e5df1cb2212f724ca7d82f304a1203b1fb711a15b0fe9556a18225527f61cd0)

就会给`From`地址mint一个ERC-721的Token

可进行`Transfer`操作[Tx](https://mumbai.polygonscan.com/token/0xe8fa8ad2d5d0bd402f8115f6e2d4e24ebca8cca7?a=48)
对LenHub产生影响的部分在交易log中有体现，但具体的数据关联还不够清晰

取消Follow操作[Tx](https://mumbai.polygonscan.com/tx/0x964c06e320484b66f7b3e9431dfef16cd364e73a511e03707499125cf7e56e00)
就是Burn掉对应的ERC-721的token

🤔️ UI-APP可以看到follows的地址，但是合约中没有找到可以一次性查询到地址下所有的follow数据


## Profile Interaction - Collect

方法`createCollect`的核心logic
```
address collectNFT = _pubByIdByProfile[rootProfileId][rootPubId].collectNFT;

if (collectNFT == address(0)) {
    collectNFT = Clones.clone(collectNFTImpl);
    _pubByIdByProfile[rootProfileId][rootPubId].collectNFT = collectNFT;

    ICollectNFT(collectNFT).initialize(
                rootProfileId,
                rootPubId,
                collectNFTName,
                collectNFTSymbol
            );

ICollectNFT(collectNFT).mint(collector);

ICollectModule(rootCollectModule).processCollect(
            profileId,
            collector,
            rootProfileId,
            rootPubId,
            collectModuleData
        );
```

## 还存在的问题

还存在的没有理解的技术实现


Q：如何实现限制性查看帖子的功能的？链上存的数据是什么？为什么满足了限制条件就能看到内容了？

Q：如何实现帖子开放/关闭“评论，转发，点赞”的功能的？

Q：如何实现App页面“like”功能的，这个并没有设计在Lens protocol的合约中？

Q：合约没有“删帖”的功能，如何去限制不适合被发布的信息内容？

Q: 合约的storage数据如何支持frontend的统计维度，eg expoler页面每个帖子的时间，comment总数，collect总数，mirror总数？

Follow Module的签名交易

```
function setFollowModule(
    uint256 profileId,
    address followModule,
    bytes calldata followModuleData
) external;


function setFollowModuleWithSig(
    DataTypes.SetFollowModuleWithSigData calldata vars
) external;
```